In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import pyGadget as pyg

In [ ]:
palette = ["#404040", "#4C72B0", "#55A868", "#C44E52", "#8172B2"]
sns.set(context='paper', style='ticks', palette=palette, font='serif', font_scale=1.6)
mpl.rc('text', usetex=True)

In [ ]:
simname = {'vanilla': r'J = 0',
        'xr_tau2_J0': r'J = J$_{0}$', 
        'xr_tau2_J1': r'J = 10 J$_{0}$', 
        'xr_tau2_J2': r'J = 10$^2$ J$_{0}$',
        'xr_tau2_J3': r'J = 10$^3$ J$_{0}$'}
simlist = ['vanilla', 'xr_tau2_J0', 'xr_tau2_J1', 'xr_tau2_J2', 'xr_tau2_J3']

In [ ]:
simV = pyg.sim.Simulation('stampede/vanilla',length='AU', mass='solar', refine=False)
sim0 = pyg.sim.Simulation('stampede/xr_tau2_J0',length='AU', mass='solar', refine=False)
sim1 = pyg.sim.Simulation('stampede/xr_tau2_J1',length='AU', mass='solar', refine=False)
sim2 = pyg.sim.Simulation('stampede/xr_tau2_J2',length='AU', mass='solar', refine=False)
nV, n0, n1, n2, tag = 355, 255, 169, 227, '_t0'
snapV = simV.load_snapshot(nV)
snap0 = sim0.load_snapshot(n0)
snap1 = sim1.load_snapshot(n1)
snap2 = sim2.load_snapshot(n2)
t0 = [snapV, snap0, snap1, snap2]
nV, n0, n1, n2, tag = 1900, 1778, 1669, 1727, '_t5k'
snapV = simV.load_snapshot(nV)
snap0 = sim0.load_snapshot(n0)
snap1 = sim1.load_snapshot(n1)
snap2 = sim2.load_snapshot(n2)
t5k = [snapV, snap0, snap1, snap2]


In [ ]:
for t, snaplist in zip([0,5000], [t0,t5k]):
    for i, snap in enumerate(snaplist):
        print t, snap.number, snap.header.Redshift

In [ ]:
rdisk = 20000 #AU

face = [[('x', 0.29518), ('z', 0.825), ('x',np.pi/2)],
        [('z', 1.16486), ('x',np.pi/2)],
        [('y', 1.87084), ('z', 2.1885), ('x',np.pi/2)],
        [('y', 0.95307), ('z', 1.5884), ('x',np.pi/2)]
       ]

disk = pd.DataFrame()
for t, snaplist in zip([0,5000], [t0,t5k]):
    for i, snap in enumerate(snaplist):
        snap.gas.cleanup()
        print '\n'+snap.sim.name, t
        pos = snap.gas.get_coords(system='spherical', centering='avg', view=face[i])
        indisk = np.where(snap.gas.spherical_coords[:,0] <= rdisk)[0]
        print '   ', indisk.size, "particles inside", rdisk, snap.sim.units.length_unit
        pos = pos[indisk]
        vel = snap.gas.get_velocities(system='spherical')[indisk]
        mass = snap.gas.get_masses()[indisk]
        dens = snap.gas.get_number_density()[indisk]
        temp = snap.gas.get_temperature()[indisk]
        data = np.column_stack((pos[:,0],vel[:,0],vel[:,2],mass,dens,temp))
        cols = ['radius', 'vr', 'vphi', 'mass', 'density', 'temperature']
        df = pd.DataFrame(data, columns=cols)
        df['sim'] = snap.sim.name.split('/')[1]
        df['time'] = t
        disk = pd.concat((disk, df))
        snap.gas.cleanup()
        snap.close()

In [ ]:
disk['cs'] = np.sqrt(pyg.constants.k_B * disk.temperature / pyg.constants.m_H)
disk['omega'] = disk.vphi / (disk.radius * 149597871)
disk['shu_acc'] = 0.975 * disk.cs**3 / pyg.constants.GRAVITY / 1.989e33 * 3.15569e7 * 1e3

In [ ]:
diskbins = np.linspace(0,1.1e4,21)
rcm = diskbins * 1.49597871e13
area = np.pi * (rcm[1:]**2 - rcm[:-1]**2)
midpoints = diskbins[:-1] + np.diff(diskbins)/2
# bin the data
grouped = disk.groupby(['time', 'sim', pd.cut(disk.radius, diskbins, labels=diskbins[:-1])])
binned = grouped.mean()
binned.rename(columns={'radius':'r_avg'}, inplace=True)
binned['mass'] = grouped['mass'].sum()
binned['area'] = np.concatenate([area]*8)
binned['sigma'] = binned.mass / binned.area * 1.989e33
binned['q'] = binned.cs * binned.omega / (np.pi * pyg.constants.GRAVITY * binned.sigma)
binned.reset_index('sim',inplace=True)
binned.reset_index('time',inplace=True)

In [ ]:
with sns.axes_style("ticks"):
    fig, ax = plt.subplots(1, 1, sharex=True, sharey=True, figsize=(8,5))

for i, time in enumerate([5000]):
    for sim in simlist[:-1]:
        df = binned[(binned.sim == sim) & (binned.time == time)]
        ax.plot(df.index,df.q,
                      ls='-', label=simname[sim], marker='o', ms=5, lw=2.5)
        ax.axhline(1.0, color='.5', zorder=1)
ax.set_ylim(.1,1.9)
ax.set_xlim(1e2, 1e4)

ax.legend(loc=0)
ax.set_ylabel('Q')
ax.set_xlabel("Radius [AU]")
fig.subplots_adjust(hspace=0.01)
sns.despine()
fig.savefig('figures/radial_bins/toomreQ.png', bbox_inches='tight')